<div style="text-align: right"> Amir Oskoui <br> Casey Tirshfield </div>
<center> <h1>Final Project - IEOR 4735</h1> </center>
<center> <h2>December 30, 2018</h2> </center>


## Objective

Our objective is to code a pricing routine for a derivative contract paying $(S_T-K)^+$ in USD at a pre-specified expiration date $T$, where $S_T$ is the price of STOXX50E denominated in EUR and K a given strike price in USD. The contract "knocks-out" if on a specified date $T_1<T$ the 3-month USD LIBOR is above a known barrier level $L^*$.
		
We simulate: $$\mathbb{E}^{\mathbb{Q}^d}\left[\left.\left(S_T-K\right)^+\cdot\left(\unicode{x1D7D9}_{L_{T_1-T_1+\delta}<L^*}\right)\cdot \left(e^{-\int_t^T r du}\right)\right\vert\mathcal{F}_t\right]$$

## Methodology

We apply two-factor Monte Carlo to simulate:
	\begin{equation*}
		\begin{cases}
		    \frac{dS}{S} = (r_f-\rho_{SX}\sigma_X\sigma_S)dt+\sigma_S dW^{\mathbb{Q}^d} \\
		    dr_d = (\Theta(t)-br)dt + \sigma_r dZ^{\mathbb{Q}^d}
	    \end{cases}
	\end{equation*}
where:

$r_f$: $\:$is the constant foreign exchange rate <br>
$\rho_{SX}$: $\:$ is the historic correlation of the stock and the foreign exchange rate <br>
$\sigma_X$: $\:$ is the foreign exchange volatility <br>
$\sigma_S$: $\:$ is the stock volatility. In the foreign currency, we will use a European option price to back out $\sigma_S$ from Black Scholes i.e. we will obtain the implied volatility using a solver because, by Girsanov's theorem, changing measure does not change volatility. <br>
$\sigma_r$: $\:$Assuming known caplet prices (which we will obtain from Bloomberg) we will use the equation for $c(t,T,K,S)$ found on page 386 of Bj$\unicode{x00f6}$rk to back out $\sigma_r$ same manner as $\sigma_S$.

## Formulas

1. Hull-White Term Structure (24.48 in Bj$\unicode{x00f6}$rk) $$p(t,T)=\frac{p^*(0,T)}{p^*(0,t)}\exp\left\{B(t,T)f^*(0,t)-\frac{\sigma^2}{4a}B^2(t,T)(1-e^{-2at})-B(t,T)r(t)\right\} $$


2. Bond Options (24.9 in Bj$\unicode{x00f6}$rk)
\begin{align*} c(t,T,K,S)&=p(t,S)N(d)-p(t,T)\cdot K \cdot N(d-\sigma_p) \end{align*}

$\hspace{1.5 em}$where,

\begin{align*} d&=\frac{1}{\sigma_p}\mathrm{log}\left\{\frac{p(t,S)}{p(t,T)K}\right\}+\frac{1}{2}\sigma_p, \\ \sigma_p &= \frac{1}{a} \left\{1-e^{-a(S-T)}\right\} \cdot \sqrt{\frac{\sigma^2}{2a}\left\{1-e^{-2a(T-t)}\right\}}\end{align*}

## Calibration

### Packages

In [1]:
import numpy as np
import math
from math import log, sqrt, exp, pi, gamma
from scipy.integrate import quad
import scipy.stats as si
from scipy.stats import norm
from scipy.special import kv
from scipy.optimize import fsolve
from scipy import interpolate
from scipy.interpolate import interp1d
import pandas as pd
from scipy.stats.distributions import norm

### Observed Market Data Variables

In [2]:
""" Price an option using the Black-Scholes model.
        a: mean rate of return can be set to 3% or something in that ballpark
        r: domestic short rate observed at time t
        t: observed time for bond
        T: matuiry of bond
        T_1: time at which LIBOR must be under L_star
        L_star: LIBOR barrier level
        B: One of the Affine term structure variables solved above
        S: maturity of LIBOR
        S_r: further maturity of caplet
        f_star: observed forward rate from the market
        P_T: observed bond price maturing at time T<S
        P_t: observed bond price maturing at time t
        K: domestic stock strike
        K_c: caplet strike
        """
a = 0.03
r = 0.028
T = 2.0
T_1 = 1.0
L_star = 0.1
S = T_1 + (3/12)
S_r = 2.0
r_f = .028 #foreign interest
K = 1.0
K_c = 1.2
## Barrier:
p_t=.999

c = 0.20     # Caplet price
p_S = 0.95   # Bond price at time S
p_T = 0.98   # Bond price at time T < S
K = 1.2      # Strike

SN = 3.00  # Option price
S_0 = 18   # Spot
T = 1.00   # Maturity
K = 20     # Strike



### Calculate $\sigma_S$

In [3]:
def f(sig_s):
    d1 = (log(S_0 / K) + (r + 0.5 * sig_s ** 2) * T) / (sig_s * sqrt(T))
    d2 = d1 - sig_s * sqrt(T)
    return si.norm.cdf(d1) * S_0 - K * exp(-r * T) * si.norm.cdf(d2) - SN

# Numerically solve for volatility
sig_s = fsolve(f, .01)
print('Asset Volatility:', float(sig_s))

Asset Volatility: 0.4978090595609679


### Calculate $\sigma_r$

In [4]:
def g(sig_r):
    sig_p = 1/a * (1 - exp(-a * (S-T))) * sqrt((sig_r**2)/(2*a) * (1 - exp(-2*a*T)))
    d = 1/sig_p * log(p_S/(p_T*K_c)) + 1/2 * sig_p
    return p_S * si.norm.cdf(d) - p_T * K * si.norm.cdf(d - sig_p) - c

# Numerically solve for volatility
sig_r = fsolve(g, .999)
print('Volatility:', float(sig_r))

Volatility: 0.048954859140369233


/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


### Calculate $\rho_{SX}$

In [5]:
X = np.random.random_sample(15)
S = np.random.random_sample(15)

In [6]:
rho = np.corrcoef(X,S)[0][1]

### Interpolation of Discount Factors

In [7]:
##Import discount factors p(0,T) as a vector
#p=csv.read()

p = np.linspace(.999, .93, num=100, endpoint=True)
x = np.linspace(0, T, len(p), endpoint=True)

p_star = interp1d(x, p, kind='cubic')
#p_star = interpolate.splrep(x, p)
#No Smoothing:

#p_star = list(interpolate.splrep(x, p, s=0))

'''
plt.figure()
plt.plot(x, p, x, p_star, '--')
print(len(p_star))
'''

"\nplt.figure()\nplt.plot(x, p, x, p_star, '--')\nprint(len(p_star))\n"

### Numerical 1st and 2nd Order Derivative of log of Discount Factors for Forward Rate and $\Theta(t)$

In [8]:
def f_star(T):
    epsilon=.000001
    f_star_T = (-log(p_star(np.array(T)))+log(p_star(np.array(T-epsilon))))/epsilon
    #neg_logp = -log(p)
    #f_star_T = interpolate.splev(x, neg_logp, der=1)
   
    return f_star_T

In [9]:
def Theta(T):
    epsilon = .000001
    #f_star_dot = interpolate.splev(x, f_star(T), der=1)
    f_star_dot = (f_star(T)-f_star(T-epsilon))/epsilon
    g_dot = ((sig_r**2)/a)*exp(-a*T)
    g = ((sig_r**2)/(2*a**2))*(1-exp(-a*T))**2
    Theta = f_star_dot + g_dot + a*(f_star(T) + g)
    return Theta

In [10]:
def B(t,T):
        """ Price an option using the Black-Scholes model.
        a: mean rate of return can be set to 3% or something in that ballpark
        t: observed time for bond
        T: Matuiry of Bond
        """
        B = (1 - exp(-(a)*(T-t)))/(a)
        
        return B

### Hull-White Term Structure for Bond Price

In [11]:
def p(t,T,r):
        """ Price an option using the Black-Scholes model.
        a: mean rate of return can be set to 3% or something in that ballpark
        t: observed time for bond
        T: Matuiry of Bond
        B: One of the Affine term structure variables solved above
        r: domestic short rate observed at time t
        f_star: observed forward rate from the market
        P_T: observed bond price maturing at time T
        P_t: observed bond price maturing at time t
        """
    
        p = (p_T/p_t)*exp(B(t,T)*f_star(t) - ((sig_r**2)/(4*a))*(B(t,T)**2)*(1-exp(-2*a*t)) - B(t,T)*r)

        return p

### Libor $L(t,T_1,S)$

In [12]:

def L(t,T_1,S,r):
    L = (p(t,T_1,r)-p(t,S,r)) / (p(t,S,r)*(S-T_1))
    return L

## Simulation

In [13]:
T = 2
T_1 = 1
s= T_1 + (3/12)
t=.01
m1 = 100
nSims = 10000
dt = T_1 / m1
dT = (T - T_1)/m1
sig_x = .03
a=.03
S = np.zeros((2*m1+1, nSims))
r = np.zeros((2*m1+1, nSims))
S[0,:] = 100
r[0,:] = 0.028
RT = np.zeros((nSims))
V = np.zeros((nSims))
I= np.zeros((nSims))

for n in range(nSims):
    for i in range(1, m1):
        Z_1 = np.random.normal(0, 1)
        Z_2 = np.random.normal(0, 1)
        S[i,n] = S[i-1,n] * exp((r_f-rho*sig_x*sig_s-1/2*sig_s**2) * dt + sig_s * sqrt(dt) * Z_1)
        r[i,n] = r[i-1,n] + (Theta(t) - B(t, T) * r[i-1, n]) * dt + sig_r * sqrt(dt) * Z_2
    if L(T_1,T_1,s, r[m1,n]) > L_star:
        I[n]=0
    else:
        I[n]=1
    for i in range(m1+1, 2*m1+1):
        Z_1 = np.random.normal(0, 1)
        Z_2 = np.random.normal(0, 1)
        S[i,n] = S[i-1,n] * exp((r_f-rho*sig_x*sig_s-1/2*sig_s**2) * dT + sig_s * sqrt(dT) * Z_1)
        r[i,n] = r[i-1,n] + (Theta(t) - B(t, T) * r[i-1, n]) * dT + sig_r * sqrt(dT) * Z_2
    for i in range(1,m1):
        RT[n]= RT[n] + (r[i,n] -r[i-1,n])*dt
    for i in range(m1+1,2*m1+1):
        RT[n]= RT[n] + (r[i,n] -r[i-1,n])*dT
    V[n] = I[n]*max(S[2*m1,n]-K,0)*exp(-RT[n])
    
V_hat = np.mean(V)

In [14]:
print(V_hat)

0.0
